In [7]:
import cv2
from ultralytics import YOLO

# モデルの読み込み。姿勢推論用のモデルデータを読み込む
model = YOLO("yolov8n-pose.pt")

pic = "federer_backhand.jpg"
frame = cv2.imread(pic)
result = model(frame)

plottedFrame = result[0].plot()
cv2.imwrite(pic + "-result.jpg", plottedFrame)


0: 352x640 1 person, 242.0ms
Speed: 11.9ms preprocess, 242.0ms inference, 21.2ms postprocess per image at shape (1, 3, 352, 640)


True

In [1]:
"Roger Federer & Tomas Berdych ｜ IW Court Level Practice.mp4"

'Roger Federer & Tomas Berdych ｜ IW Court Level Practice.mp4'

In [2]:
pip install imageio

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import imageio

In [2]:

import cv2
import numpy as np
from ultralytics import YOLO

# モデルの読み込み。姿勢推論用のモデルデータを読み込む
model = YOLO("yolov8n-pose.pt")

# 入力ビデオファイルのパス
input_video = "/Users/yusuke.s/Documents/pickleball_videos/forehand_volley/xfhjsg.mp4"

# 出力ビデオファイルのパス
output_video = "output_video.mp4"

# ビデオキャプチャオブジェクトの作成
cap = cv2.VideoCapture(input_video)

# ビデオの属性を取得
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 出力ビデオライターの設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# 座標データを保存するリスト
all_keypoints = []

frame_count = 0
while cap.isOpened():
    # フレームを読み込む
    success, frame = cap.read()
    if not success:
        break

    # YOLOモデルで推論を行う
    results = model(frame)

    # 結果を描画したフレームを取得
    annotated_frame = results[0].plot()

    # キーポイントの座標を取得
    if len(results[0].boxes) > 0:
        # バウンディングボックスの面積を計算
        areas = [(box.xyxy[0][2] - box.xyxy[0][0]) * (box.xyxy[0][3] - box.xyxy[0][1]) for box in results[0].boxes]
        # 最大面積を持つバウンディングボックスのインデックスを取得
        largest_box_index = np.argmax(areas)
        
        keypoints = results[0].keypoints[largest_box_index]  # 最大の長方形に対応するキーポイント
        
        # デバッグ出力
        print(f"Frame {frame_count} keypoints shape: {keypoints.xy.shape}")
        print(f"Keypoints data: {keypoints.xy}")

        # キーポイントが2次元配列であり、少なくとも2つの値を持つことを確認
        if keypoints.xy.ndim == 3 and keypoints.xy.shape[2] >= 2:
            all_keypoints.append(keypoints.xy[0].cpu().numpy())  # 最大の長方形の人物のキーポイントのみを保存

            # キーポイントを描画（オプション）
            for point in keypoints.xy[0]:
                x, y = point[:2]
                cv2.circle(annotated_frame, (int(x), int(y)), 5, (0, 255, 0), -1)
        else:
            print(f"Warning: Invalid keypoint data in frame {frame_count}")

    # 処理したフレームを出力ビデオに書き込む
    out.write(annotated_frame)

    # 処理状況を表示（オプション）
    cv2.imshow("YOLOv8 Inference", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_count += 1

# リソースの解放
cap.release()
out.release()
cv2.destroyAllWindows()

# 座標データをNumPy配列として保存
if all_keypoints:
    np.save('keypoints_data.npy', np.array(all_keypoints))
    print(f"Processed {frame_count} frames")
    print(f"Keypoints data saved to keypoints_data.npy")
    
    # キーポイントデータの形状を確認
    keypoints_array = np.array(all_keypoints)
    print(f"Keypoints data shape: {keypoints_array.shape}")
else:
    print("No valid keypoints data was collected.")


0: 384x640 3 persons, 371.8ms
Speed: 10.8ms preprocess, 371.8ms inference, 30.1ms postprocess per image at shape (1, 3, 384, 640)


Frame 0 keypoints shape: torch.Size([1, 17, 2])
Keypoints data: tensor([[[  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [558.8593, 223.6208],
         [596.8178, 224.9405],
         [525.6777, 269.9388],
         [614.7839, 272.6838],
         [506.8005, 335.6578],
         [626.2600, 343.6624],
         [531.6067, 362.1826],
         [614.4081, 380.4658],
         [536.7409, 398.5438],
         [598.7604, 398.3880],
         [540.2319, 510.3312],
         [613.4751, 505.3246],
         [535.3020, 613.5496],
         [624.7803, 608.9522]]])



0: 384x640 3 persons, 213.4ms
Speed: 2.4ms preprocess, 213.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)



Frame 1 keypoints shape: torch.Size([1, 17, 2])
Keypoints data: tensor([[[  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [597.2088, 218.4986],
         [526.5095, 266.0024],
         [614.4534, 267.5851],
         [507.5121, 333.5380],
         [625.0742, 340.5450],
         [530.0820, 357.2502],
         [613.3513, 379.6679],
         [537.9793, 393.7163],
         [599.9675, 393.3109],
         [534.9089, 505.1368],
         [613.1331, 501.8865],
         [527.1289, 607.6443],
         [625.8493, 606.9992]]])


0: 384x640 3 persons, 197.0ms
Speed: 1.2ms preprocess, 197.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)



...

Frame 51 keypoints shape: torch.Size([1, 17, 2])
Keypoints data: tensor([[[  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [390.8613, 239.5162],
         [321.7008, 287.9201],
         [412.5023, 287.5534],
         [317.3351, 357.6423],
         [429.9297, 363.8241],
         [335.9799, 368.4180],
         [410.9372, 390.6522],
         [329.3822, 410.5933],
         [395.0926, 410.5549],
         [317.6501, 508.7718],
         [402.0670, 512.6833],
         [289.1879, 604.2771],
         [397.4150, 614.0795]]])


0: 384x640 2 persons, 185.9ms
Speed: 1.1ms preprocess, 185.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)



Frame 52 keypoints shape: torch.Size([1, 17, 2])
Keypoints data: tensor([[[  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [397.7150, 237.0823],
         [331.1967, 286.1335],
         [416.8564, 286.2580],
         [321.0513, 354.7462],
         [435.9792, 360.0264],
         [336.5098, 367.0126],
         [413.9343, 385.3935],
         [336.3339, 411.7995],
         [397.1326, 410.5105],
         [325.5508, 508.7365],
         [402.2129, 510.5713],
         [301.4832, 602.4168],
         [392.8863, 612.2952]]])


0: 384x640 2 persons, 192.7ms
Speed: 1.3ms preprocess, 192.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


Frame 53 keypoints shape: torch.Size([1, 17, 2])
Keypoints data: tensor([[[  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [399.9077, 233.6383],
         [336.5876, 281.6636],
         [419.2255, 281.6190],
         [324.0275, 348.2240],
         [437.7485, 354.7332],
         [336.2057, 366.5742],
         [418.4501, 386.2067],
         [343.4452, 407.1776],
         [400.6390, 405.6442],
         [333.8528, 508.1288],
         [399.8238, 508.8639],
         [314.4912, 606.0416],
         [389.6958, 614.0881]]])
Processed 54 frames
Keypoints data saved to keypoints_data.npy
Keypoints data shape: (54, 17, 2)
